In [1]:
from edinburgh_challenge.constants import police_stations, police_stations_dict

In [2]:
from dataclasses import dataclass
from typing import List, Dict
from abc import ABC, abstractmethod

import numpy as np
import pandas as pd
from math import radians, cos, sin, asin

from edinburgh_challenge.constants import Shift
from edinburgh_challenge.simulation import Simulator
from edinburgh_challenge.models import GreedyModel



class Model(ABC):
    @abstractmethod
    def make_allocation(self, incidents, officers, current_time):
        pass

@dataclass
class Incident:
    urn: str
    latitude: float
    longitude: float
    day:int
    hour:int
    global_time: float  # Time at which the incident was reported (24*day + hour)
    deployment_time: float  # Time taken to resolve the incident after reaching the location
    priority: str
    distances: Dict[str, float]  # Distances from each station
    resolved: bool = False
    resolving_officer: str = None
    response_time: float = None  # Global time when the response arrives at the scene
    resolution_time: float = None  # Global time when the incident is resolved
    allocation_time:float = None

@dataclass
class Officer:
    name: str
    station: str
    available: bool = True
    return_time: float = 0.0


# Shift distribution structure
# Example: {'Early': {'Station_1': 5, 'Station_2': 5, 'Station_3': 5}, ...}
ShiftDistribution = Dict[str, Dict[str, int]]

weekday_shift_distribution = {
    'Early': 20,
    'Day': 50,
    'Night':30
}

weekend_shift_distribution = {
    'Early': 20,
    'Day': 40,
    'Night': 40
}

In [3]:
source = "./data.xlsx"
data = pd.read_excel(source)
data["Time"] = (data["Day"]-1)*24 + data["Hour"]
data.columns = [x.lower() for x in data.columns]

ps = [[f'PS_{i}', p.x, p.y] for i,p in 
      enumerate([police_stations.one, 
                 police_stations.two, 
                 police_stations.three])]
df_ps = pd.DataFrame(ps, columns=["Name","Latitude", "Longitude"])

## Shift Distibution Change

In [4]:
shift_distribution = {'Early': {'Station_1': 0, 'Station_2': 5, 'Station_3': 10},
  'Day': {'Station_1': 2, 'Station_2': 19, 'Station_3': 4},
  'Night': {'Station_1': 9, 'Station_2': 3, 'Station_3': 28}
}


ps_coords = [ (p.x, p.y) for p in 
                [police_stations.one, 
                 police_stations.two, 
                 police_stations.three]]

simulation = Simulator(data, ps_coords,shift_distribution, verbose=-1)



greedy_model = GreedyModel(shift_distribution, police_stations_dict)

In [6]:
# No. of officers per shift
simulation.run(greedy_model)
simulation.analyze_simulation_results()

ZeroDivisionError: division by zero

In [ ]:
simulation.analyze_simulation_results()